In [144]:
import pandas as pd
import numpy as np

In [145]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [146]:
movies = movies.merge(credits, on='title')

In [147]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [148]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [149]:
import ast
def convert(obj):
  l = []
  for i in ast.literal_eval(obj):
    l.append(i['name'])
  return l

In [150]:
movies.dropna(inplace=True)

<ipython-input-150-4d2d825739df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.dropna(inplace=True)


In [151]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [152]:
movies.duplicated().sum()

0

In [153]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [154]:
movies['genres'] = movies['genres'].apply(convert)

In [155]:
movies['keywords'] = movies['keywords'].apply(convert)

In [156]:
def convert1(obj):
  l = []
  counter = 0
  for i in ast.literal_eval(obj):
    if(counter<3):
      l.append(i['name'])
      counter+=1
    else:
      break
  return l

In [157]:
movies['cast'] = movies['cast'].apply(convert1)

In [158]:
import ast
def fetchDirector(obj):
  l = []
  for i in ast.literal_eval(obj):
    if(i['job'] == 'Director'):
      l.append(i['name'])
      break
  return l

In [159]:
movies['crew'] = movies['crew'].apply(fetchDirector)

In [160]:
movies.head(3)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]


In [161]:
movies['overview'] = movies['overview'].apply(lambda x : x.split())

In [162]:
movies['genres'] = movies['genres'].apply(lambda x :[i.replace(" ","")for i in x])
movies['keywords']  = movies['keywords'].apply(lambda x :[i.replace(" ","")for i in x])
movies['cast'] = movies['cast'].apply(lambda x :[i.replace(" ","")for i in x])
movies['crew'] = movies['crew'].apply(lambda x :[i.replace(" ","")for i in x])

In [163]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [164]:
new_df = movies[['movie_id', 'title', 'tags']]

In [165]:
new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))

<ipython-input-165-fe97dfee7549>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))


In [166]:
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

<ipython-input-166-574ea7a8c2f8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())


In [167]:
!pip install nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [168]:
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [169]:
new_df['tags'] = new_df['tags'].apply(stem)

<ipython-input-169-be18a4346d89>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [170]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = "english")

In [171]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [172]:
from sklearn.metrics.pairwise import cosine_similarity

In [173]:
similarity = cosine_similarity(vectors)

In [174]:
sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x : x[1])[1:6]

[(1216, 0.28676966733820225),
 (2409, 0.26901379342448517),
 (3730, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.25038669783359574)]

In [175]:
def recommend(movie):
  movie_index = new_df[new_df['title'] == movie].index[0]
  distances = similarity[movie_index]
  movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x : x[1])[1:6]

  for i in movies_list:
    print(new_df.iloc[i[0]].title)

In [176]:
import pickle

In [177]:
pickle.dump(new_df, open('movies.pkl','wb'))

In [178]:
pickle.dump(new_df.to_dict(), open('moviesDict.pkl','wb'))

In [179]:
pickle.dump(similarity, open('similarity.pkl','wb'))